In [11]:
#importing essential libraries for pyspark and sql
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
import mysql.connector
import pandas as pd

#create spark session
spark = SparkSession.builder.config("spark.jars", "/home/hduser/mysql-connector-java-5.1.47/mysql-connector-java-5.1.47.jar") \
    .master("local").appName("ETL to SQL").getOrCreate()
spark

In [12]:
#connecting to localhost sql database
 
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database ='etl'
)
 
# Creating an instance of 'cursor' class
# which is used to execute the 'SQL'
# statements in 'Python'
cursor = mydb.cursor()
 
# Show database tables
cursor.execute("SHOW TABLES")
 
for x in cursor:
  print(x)

#cursor.execute("INSERT INTO crypto (Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap) values('test',0,0,0,0,0,0,0,0)")
#mydb.commit()

('bitcoin',)
('bitcoin1',)
('cardano',)
('crypto',)
('ethereum',)
('newcrypto',)
('newcrypto1',)
('polkadot',)


In [13]:
def sql_upload(df,table):
    pd_dataframe=df.toPandas()
    for index, row in pd_dataframe.iterrows():
        cursor.execute("INSERT INTO "+table+" (date,high,low,open,close) values(%s,%s,%s,%s,%s)",(row['Date'],row['High'],row['Low'],row['Open'],row['Close']))
        mydb.commit()
    print("Successfully uploaded to sql table")

In [14]:
btc_df = spark.read.csv('csv\\bitcoin.csv', header=True, sep=",").cache()
btc_df.show()

+----------+-----+-----+-----+-----+-------+--------+
|      Date| Open| High|  Low|Close| Volume|Currency|
+----------+-----+-----+-----+-----+-------+--------+
|2020-08-21|2.909|3.078|2.909|3.078|1066769|     USD|
|2020-08-22|3.078|4.464| 2.97|4.464|4020158|     USD|
|2020-08-23|4.451|4.451|3.915|3.977|1560250|     USD|
|2020-08-24|3.977| 4.73|3.977|4.594|1559675|     USD|
|2020-08-25|4.595| 5.55|4.595| 5.55|3024350|     USD|
|2020-08-26| 5.55|6.391| 5.55|6.059|2112266|     USD|
|2020-08-27|6.059| 6.57|5.652|5.652|2218474|     USD|
|2020-08-28|5.652|6.206|5.652| 6.17|1605011|     USD|
|2020-08-29| 6.17|6.291| 6.15| 6.15| 803570|     USD|
|2020-08-30| 6.15| 6.15| 5.89| 5.89| 830835|     USD|
|2020-08-31|5.898| 6.29|5.898| 6.29| 805931|     USD|
|2020-09-01| 6.29|6.627|6.257|6.257|1016741|     USD|
|2020-09-02|6.257|6.287|5.927|6.127| 988583|     USD|
|2020-09-03|6.127|6.249|5.724|5.724|1228432|     USD|
|2020-09-04|5.724|5.893|5.102|5.124|1660662|     USD|
|2020-09-05|5.124|5.124|4.13

In [15]:
sql_upload(btc_df,"bitcoin1")
print("test")

Successfully uploaded to sql table
test


In [30]:
eth_df = spark.read.csv('csv\\ethereum.csv', header=True, sep=",").cache()
ada_df = spark.read.csv('csv\\cardano.csv', header=True, sep=",").cache()
polka_df = spark.read.csv('csv\\polkadot.csv', header=True, sep=",").cache()

In [31]:
sql_upload(eth_df,"ethereum")
sql_upload(ada_df,"cardano")
sql_upload(polka_df,"polkadot")

Successfully uploaded to sql table
Successfully uploaded to sql table
Successfully uploaded to sql table
